In [27]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import mutual_info_regression
from sklearn.linear_model import LassoCV
from sklearn.ensemble import RandomForestRegressor

Define a helper function to compute the Variance Inflation Factor (VIF) for detecting multicollinearity.

In [28]:
# VIF helper (uses statsmodels if available)
def compute_vif(df, features):
    from statsmodels.stats.outliers_influence import variance_inflation_factor
    X = df[features].assign(constant=1.0)
    vif_data = {f: variance_inflation_factor(X.values, i) for i, f in enumerate(X.columns) if f != 'constant'}
    return pd.Series(vif_data).sort_values(ascending=False)

In [29]:
df = pd.read_csv("dataset/hn_daily.csv")
df['datetime'] = pd.to_datetime(df['datetime'])
df = df.sort_values('datetime').reset_index(drop=True)
df.head()

,name,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,solarenergy,uvindex,severerisk,sunrise,sunset,moonphase,conditions,description,icon,stations
0,Hanoi,2015-01-01,24.0,11.4,17.7,24.0,11.4,17.7,9.4,63.8,...,17.1,7,NaN,2015-01-01T06:33:34,2015-01-01T17:26:18,0.36,Clear,Clear conditions throughout the day.,clear-day,"48820099999,48823099999,48825099999,4883109999..."
1,Hanoi,2015-01-02,22.0,11.0,16.3,22.0,11.0,16.3,9.4,65.6,...,16.2,7,NaN,2015-01-02T06:33:53,2015-01-02T17:26:56,0.39,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"48820099999,48823099999,48825099999,4883109999..."
2,Hanoi,2015-01-03,21.0,13.1,17.0,21.0,13.1,17.0,12.4,75.6,...,9.9,4,NaN,2015-01-03T06:34:11,2015-01-03T17:27:34,0.43,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"48820099999,48823099999,48825099999,4883109999..."
3,Hanoi,2015-01-04,22.6,16.9,19.3,22.6,16.9,19.3,16.1,82.0,...,5.6,3,NaN,2015-01-04T06:34:28,2015-01-04T17:28:12,0.46,"Rain, Partially cloudy",Partly cloudy throughout the day with a chance...,rain,"48820099999,48823099999,48825099999,4883109999..."
4,Hanoi,2015-01-05,23.0,18.5,20.4,23.0,18.5,20.4,18.4,88.4,...,7.9,4,NaN,2015-01-05T06:34:44,2015-01-05T17:28:51,0.50,Overcast,Cloudy skies throughout the day.,cloudy,"48820099999,48823099999,48825099999,4883109999..."


### 1. Data Cleaning and Preprocessing

This step removes or transforms features that could leak future information (data leakage) or are irrelevant to prediction. For example, "tempmax" and "tempmin" directly define the target variable "temp", while "conditions" or "icon" describe outcomes after the event. Columns like "sunrise" and "sunset" are transformed into a numeric "day_length" feature. This ensures the dataset reflects only information available at prediction time.

In [30]:
# Remove columns causing data leakage or non-informative
cols_to_drop = ['tempmax', 'tempmin', 'description', 'conditions', 'icon', 'name', 'stations', 'source']
df = df.drop(columns=[c for c in cols_to_drop if c in df.columns], errors='ignore')

# If sunrise/sunset are full datetimes or HH:MM:SS this will work
sr = pd.to_datetime(df['sunrise'], errors='coerce')
ss = pd.to_datetime(df['sunset'], errors='coerce')
# Compute difference in seconds, convert to hours
df['day_length_h'] = (ss - sr).dt.total_seconds() / 3600
# Drop raw sunrise/sunset
df = df.drop(columns=['sunrise','sunset'])

### 2. Temporal Feature Engineering

Extract time-based features such as month, day of year, and weekday, and encode them using sine/cosine transformations to capture cyclical seasonality (e.g., December close to January). This helps the model understand annual and weekly cycles typical in climate data.

Create lag features (e.g., yesterday’s temperature, last week’s humidity) and rolling-window statistics such as moving averages or max windspeed to help the model capture trends and short-term weather persistence.

In [31]:
# Extract datetime components
df['month'] = df['datetime'].dt.month
df['day_of_year'] = df['datetime'].dt.dayofyear
df['day_of_week'] = df['datetime'].dt.weekday
df['week_of_year'] = df['datetime'].dt.isocalendar().week.astype(int)
df['is_weekend'] = df['day_of_week'].isin([5,6]).astype(int)

# Cyclical encodings
df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
df['day_sin'] = np.sin(2 * np.pi * df['day_of_year'] / 365)
df['day_cos'] = np.cos(2 * np.pi * df['day_of_year'] / 365)

# Lag features
USE_LAGS = {
    'temp': [1,2,3,7],
    'feelslike': [1,3],
    'humidity': [1],
    'sealevelpressure': [3],
    'windspeed': [1,2]
}

for base_col, lags in USE_LAGS.items():
    if base_col in df.columns:
        for lag in lags:
            df[f'{base_col}_lag{lag}'] = df[base_col].shift(lag)

# Rolling window features
if 'precip' in df.columns:
    df['precip_roll_mean_7'] = df['precip'].shift(1).rolling(7).mean()
if 'humidity' in df.columns:
    df['humidity_roll_mean_7'] = df['humidity'].shift(1).rolling(7).mean()
if 'windspeed' in df.columns:
    df['windspeed_roll_max_3'] = df['windspeed'].shift(1).rolling(3).max()

### 3. Creating Interaction and New Features

Build physically meaningful interaction terms (e.g., temperature × humidity) and non-linear transformations (e.g., squared windspeed). These help linear models capture complex weather effects like wind chill or heat index.

In [32]:
if {'feelslikemax','humidity'}.issubset(df.columns):
    df['feelslike_humidity_int'] = df['feelslikemax'] * df['humidity']

if {'feelslikemin','windspeed'}.issubset(df.columns):
    df['windchill_int'] = df['feelslikemin'] * df['windspeed']

if 'windspeed' in df.columns:
    df['windspeed_sq'] = df['windspeed'] ** 2

if 'pressure' in df.columns and 'humidity' in df.columns:
    df['pressure_humidity'] = df['pressure'] * df['humidity']

if 'day_length_h' in df.columns and 'uvindex' in df.columns:
    df['daylength_uv'] = df['day_length_h'] * df['uvindex']

### 4. Categorical Feature Encoding

Convert categorical data (e.g., precipitation type) into numerical format using one-hot encoding so that models can process them correctly. Each category (rain, snow, ice, etc.) becomes its own binary column.

In [33]:
cat_cols = []
if 'preciptype' in df.columns:
    cat_cols.append('preciptype')

if cat_cols:
    enc = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    enc_arr = enc.fit_transform(df[cat_cols].astype(str))
    enc_cols = []
    for col, cats in zip(cat_cols, enc.categories_):
        enc_cols += [f"{col}_{c}" for c in cats]
    df_enc = pd.DataFrame(enc_arr, columns=enc_cols, index=df.index)
    df = pd.concat([df, df_enc], axis=1)
    df = df.drop(columns=cat_cols)
    print('One-hot encoded:', enc_cols)
else:
    print('No categorical columns to one-hot encode (preciptype absent)')

One-hot encoded: ['preciptype_nan', 'preciptype_rain']


Define the prediction target — the average temperature 5 days ahead — by shifting the "temp" column by −5 days. Then, drop rows with NaN values introduced by lag or rolling operations to finalize a clean dataset for training.

In [34]:
# Target creation (5-day ahead)
df['target_temp_5d'] = df['temp'].shift(-5)

# Drop rows with NaN introduced by shifts/rollings
n_before = df.shape[0]
df = df.dropna().reset_index(drop=True)
print(f'Dropped {n_before - df.shape[0]} rows due to NaNs from lags/rolling/target')

Dropped 2571 rows due to NaNs from lags/rolling/target


### 5. Feature Selection

Rank features by their importance using several approaches:
- Correlation: linear relationships with the target.
- Mutual Information: captures non-linear dependencies.
- LassoCV: automatically selects relevant features via regularization.
- RandomForest: model-based importance from tree ensembles.

Finally, aggregate ranks and save the top ~25 features.

In [35]:
X = df.drop(columns=['datetime','target_temp_5d','temp'], errors='ignore')
y = df['target_temp_5d']

# Pearson correlation with target (absolute)
corrs = X.corrwith(y).abs().sort_values(ascending=False)
print('\nTop 15 features by abs(correlation) with target:')
print(corrs.head(15))

# Mutual Information (captures non-linear relationships)
mi = mutual_info_regression(X.fillna(0), y)
mi_series = pd.Series(mi, index=X.columns).sort_values(ascending=False)
print('\nTop 15 features by mutual information:')
print(mi_series.head(15))

# VIF (optional - requires statsmodels)
vif = compute_vif(X.fillna(0), X.columns.tolist())
if vif is not None:
    print('\nTop VIF values:')
    print(vif.head(15))

# LassoCV for embedded selection
lasso = LassoCV(cv=5, random_state=42, n_jobs=-1)
lasso.fit(X.fillna(0), y)
coef_abs = pd.Series(np.abs(lasso.coef_), index=X.columns).sort_values(ascending=False)
print('\nTop 15 features by absolute Lasso coefficients:')
print(coef_abs.head(15))

# RandomForest importance
rf = RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1)
rf.fit(X.fillna(0), y)
rf_imp = pd.Series(rf.feature_importances_, index=X.columns).sort_values(ascending=False)
print('\nTop 15 features by RandomForest importance:')
print(rf_imp.head(15))

# Aggregate a simple ranking (average rank across methods)
rank_df = pd.DataFrame({
    'corr_rank': corrs.rank(ascending=False),
    'mi_rank': mi_series.rank(ascending=False),
    'lasso_rank': coef_abs.rank(ascending=False),
    'rf_rank': rf_imp.rank(ascending=False)
}).fillna(1e6) # missing treated as low importance

rank_df['mean_rank'] = rank_df.mean(axis=1)
ranked = rank_df.sort_values('mean_rank')

print('\nTop 20 features by aggregated mean rank:')
print(ranked.head(20))

# Save selected top features
selected_features = ranked.head(25).index.tolist()
print('\nSelected features (top 25):', selected_features)


Top 15 features by abs(correlation) with target:
day_cos                   0.829138
day_length_h              0.780375
feelslike                 0.766245
temp_lag1                 0.763751
temp_lag7                 0.757673
feelslikemax              0.756583
feelslike_lag1            0.753861
temp_lag2                 0.753134
temp_lag3                 0.749957
feelslike_lag3            0.743489
month_cos                 0.735211
feelslikemin              0.731395
feelslike_humidity_int    0.719746
dew                       0.715574
sealevelpressure          0.674667
dtype: float64


c:\Users\PHUC\anaconda3\envs\ml_env\Lib\site-packages\numpy\lib\_function_base_impl.py:2922: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\PHUC\anaconda3\envs\ml_env\Lib\site-packages\numpy\lib\_function_base_impl.py:2923: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]



Top 15 features by mutual information:
day_of_year       0.809054
week_of_year      0.795906
day_cos           0.697992
month             0.664845
day_length_h      0.587159
feelslike         0.497839
feelslike_lag1    0.482769
temp_lag1         0.479368
temp_lag3         0.479049
temp_lag2         0.475829
feelslike_lag3    0.472508
feelslikemax      0.461718
temp_lag7         0.451201
feelslikemin      0.449052
dew               0.437709
dtype: float64


c:\Users\PHUC\anaconda3\envs\ml_env\Lib\site-packages\statsmodels\regression\linear_model.py:1782: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss
c:\Users\PHUC\anaconda3\envs\ml_env\Lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)



Top VIF values:
preciptype_rain                   inf
preciptype_nan                    inf
solarenergy               4674.032339
solarradiation            4665.747751
day_cos                   1962.087949
day_length_h              1898.976279
daylength_uv               520.383668
uvindex                    455.027451
feelslikemax               324.006046
feelslike_humidity_int     240.764151
day_of_year                171.379454
month                      150.248487
feelslike                  149.003422
dew                        117.515670
day_sin                    105.141531
dtype: float64

Top 15 features by absolute Lasso coefficients:
temp_lag7                 0.134993
feelslike_lag3            0.060153
precip_roll_mean_7        0.016418
precip                    0.015951
daylength_uv              0.010605
cloudcover                0.008642
windchill_int             0.007747
windspeed_sq              0.005057
day_of_year               0.002188
feelslike_humidity_int    0.002013